In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

print("✅ Libraries imported successfully!")

c:\Users\user\Documents\working with LLM\benv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported successfully!


In [2]:
# Load a small, fast embedding model
print("Loading embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded!")
print(f"Model produces {model.get_sentence_embedding_dimension()} dimensional embeddings")

Loading embedding model...
✅ Model loaded!
Model produces 384 dimensional embeddings


In [3]:
sentences = [
    "The dog is playing in the park",
    "A puppy is running outside",
    "The cat is sleeping on the couch",
    "Python is a programming language",
    "Machine learning models need data",
    "I love coding in Python"
]

In [4]:

# Generate embedding
embedding = model.encode(sentences)

print(f"Original text: {sentences}")
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding type: {type(embedding)}")
print(f"\nFirst 10 values: {embedding[:10]}")

Original text: ['The dog is playing in the park', 'A puppy is running outside', 'The cat is sleeping on the couch', 'Python is a programming language', 'Machine learning models need data', 'I love coding in Python']
Embedding shape: (6, 384)
Embedding type: <class 'numpy.ndarray'>

First 10 values: [[ 0.04757565 -0.07015254  0.06429745 ...  0.07358797  0.01249521
   0.01645609]
 [-0.02525662  0.0305436   0.05249953 ...  0.01063143  0.01476686
   0.10832039]
 [ 0.12203885 -0.0475138  -0.00115908 ...  0.08472569  0.06573962
   0.00923305]
 [-0.0353708   0.03816499 -0.04126011 ...  0.11130317  0.19625434
  -0.02897431]
 [ 0.01665931 -0.0455886   0.0234651  ...  0.02717926 -0.03379649
  -0.05370042]
 [-0.06430854  0.01564188 -0.0467849  ...  0.15115136  0.10791416
  -0.04270951]]


In [5]:
def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between two vectors.
    
    Returns a score between -1 and 1 (higher = more similar)
    """
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

print("✅ Similarity function ready!")

✅ Similarity function ready!


In [7]:
# Create test sentences
sentences = [
     "The dog is playing in the park",
    "A puppy is running outside",
    "The cat is sleeping on the couch",
    "Python is a programming language",
    "Machine learning models need data",
    "I love coding in Python"
]

# Generate embeddings for all sentences
embeddings = model.encode(sentences)

# Compare first sentence to all others
print("Comparing to: 'The dog is playing in the park'\n")
for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[0], embeddings[i])
    print(f"Similarity to '{sentence}'")
    print(f"Score: {similarity:.3f}\n")

Comparing to: 'The dog is playing in the park'

Similarity to 'The dog is playing in the park'
Score: 1.000

Similarity to 'A puppy is running outside'
Score: 0.398

Similarity to 'The cat is sleeping on the couch'
Score: 0.071

Similarity to 'Python is a programming language'
Score: 0.099

Similarity to 'Machine learning models need data'
Score: -0.005

Similarity to 'I love coding in Python'
Score: 0.090



In [9]:
# Create test sentences
sentences = [
     "The dog is playing in the park",
    "A puppy is running outside",
    "The cat is sleeping on the couch",
    "Python is a programming language",
    "Machine learning models need data",
    "I love coding in Python"
]

# Generate embeddings for all sentences
embeddings = model.encode(sentences)

# Compare first sentence to all others
print("Comparing to: 'Python is a programming language'\n")
for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[3], embeddings[i])
    print(f"Similarity to '{sentence}'")
    print(f"Score: {similarity:.3f}\n")

Comparing to: 'Python is a programming language'

Similarity to 'The dog is playing in the park'
Score: 0.099

Similarity to 'A puppy is running outside'
Score: 0.040

Similarity to 'The cat is sleeping on the couch'
Score: 0.020

Similarity to 'Python is a programming language'
Score: 1.000

Similarity to 'Machine learning models need data'
Score: 0.113

Similarity to 'I love coding in Python'
Score: 0.730



Excercise 2

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between two vectors.
    Returns a score between -1 and 1 (higher = more similar)
    """
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

class SimpleRetriever:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initialize retriever with embedding model.
        """
        self.model = SentenceTransformer(model_name)
        self.chunks = []
        self.embeddings = None

    def add_document(self, document, chunk_size):
        """
        Chunk a single document and add its chunks and embeddings to the retriever.
        """
        self.chunks = []
        for i in range(0, len(document), chunk_size):
            chunk = document[i:i+chunk_size]
            chunk = " ".join(chunk.strip().split())
            if chunk:
                self.chunks.append(chunk)
        
        # Generate embeddings
        self.embeddings = self.model.encode(self.chunks)

    def search(self, query, top_k=3):
        """
        Search for relevant chunks.
        """
        query_embedding = self.model.encode(query)
        
        # Calculate similarities
        similarities = []
        for i, chunk_emb in enumerate(self.embeddings):
            sim = cosine_similarity(query_embedding, chunk_emb)
            similarities.append((self.chunks[i], sim))
        
        # Sort and return top k
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

document = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to
the natural intelligence displayed by humans and animals. Leading AI textbooks define
the field as the study of intelligent agents: any device that perceives its environment
and takes actions that maximize its chance of successfully achieving its goals.

Machine learning is a subset of artificial intelligence that focuses on the use of data
and algorithms to imitate the way that humans learn, gradually improving its accuracy.
Machine learning is an important component of the growing field of data science.

Deep learning is part of a broader family of machine learning methods based on artificial
neural networks with representation learning. Learning can be supervised, semi-supervised
or unsupervised. Deep learning architectures such as deep neural networks, deep belief
networks, recurrent neural networks and convolutional neural networks have been applied
to fields including computer vision, speech recognition, natural language processing,
machine translation, and bioinformatics.

Natural language processing is a subfield of linguistics, computer science, and artificial
intelligence concerned with the interactions between computers and human language, in
particular how to program computers to process and analyze large amounts of natural
language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation.
"""

query = "What is machine learning?"
retriever = SimpleRetriever()
chunk_sizes = [100, 200, 400]
results_summary = {}

print("Chunk Size Comparison:\n")

for size in chunk_sizes:
    retriever.add_document(document, chunk_size=size)
    results = retriever.search(query, top_k=3)
    
    print(f"{'='*30}")
    print(f"Chunk Size: {size} characters")
    print(f"{'='*30}")
    print(f"- Number of chunks: {len(retriever.chunks)}")
    
    if results:
        top_chunk, top_score = results[0]
        print(f"- Top result: \"{top_chunk}\"")
        print(f"- Score: {top_score:.3f}")
        
        analysis = ""
        if size == 100:
            analysis = "Chunks are too small, cutting sentences mid-thought. The top result captures the phrase 'Machine learning is' but lacks the full definition, leading to a less focused answer."
        elif size == 200:
            analysis = "This size captures complete or near-complete sentences containing the core definition of machine learning. The top result provides a clear, focused answer to the query."
        elif size == 400:
            analysis = "The chunk is larger and contains the definition, but it also includes less relevant information about data science. While complete, it is less focused than the medium chunk."
        print(f"- Analysis: {analysis}\n")
        
        results_summary[size] = {"top_chunk": top_chunk, "score": top_score}
    else:
        print("- No results found.\n")

# --- Final Comparison ---

print("Best chunk size for this use case: **200 characters**")
print("because it provided the most **focused** answer. The chunk size was sufficient to capture the complete definition of 'machine learning' in a single chunk without including too much extraneous information, as was the case with the larger chunk size. The smaller chunks were too fragmented to provide a complete answer.")

Chunk Size Comparison:

Chunk Size: 100 characters
- Number of chunks: 16
- Top result: "nce of successfully achieving its goals. Machine learning is a subset of artificial intelligence th"
- Score: 0.662
- Analysis: Chunks are too small, cutting sentences mid-thought. The top result captures the phrase 'Machine learning is' but lacks the full definition, leading to a less focused answer.

Chunk Size: 200 characters
- Number of chunks: 8
- Top result: "at focuses on the use of data and algorithms to imitate the way that humans learn, gradually improving its accuracy. Machine learning is an important component of the growing field of data science. D"
- Score: 0.693
- Analysis: This size captures complete or near-complete sentences containing the core definition of machine learning. The top result provides a clear, focused answer to the query.

Chunk Size: 400 characters
- Number of chunks: 4
- Top result: "at focuses on the use of data and algorithms to imitate the way that humans learn